In [1]:
from langchain_core.tools import tool
from langgraph.graph import MessagesState
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

load_dotenv()

def reducer(a,b):
    print("A:: ",a)
    print("B:: ",b)
    if not a:
        return b
    
    return [a,b]

class State(TypedDict):
    id: str
    messages: Annotated[list, add_messages]

In [2]:
@tool
def add(a:int, b:int) -> int | None:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """

    if not a or not b:
        return None
    
    return a + b

@tool
def subs(a:int, b:int) -> int | None:
    """Substract a and b.

    Args:
        a: first int
        b: second int
    """

    if not a or not b:
        return None
    
    return a - b

@tool
def multipy(a:int, b:int) -> int | None:
    """Multipy a and b after multiplying add 50.

    Args:
        a: first int
        b: second int
    """

    if not a or not b:
        return None
    
    return a * b

@tool
def divide(a:int, b:int) -> int | None:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """

    if not a or not b:
        return None
    
    return a / b

tools = [add, subs, multipy, divide]
tools_by_name = {tool.name:tool for tool in tools}

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [3]:
builder = StateGraph(State)


def call_llm(state:State):
    print("CALL_LLM:: ",state)
    result = llm_with_tools.invoke(state["messages"])

    return {
        "messages" : result
    }
    

def should_call_node(state):
    print("SHOULD_CALL_NODE:: ",state)
    messages = state["messages"][-1]

    if messages.tool_calls:
        return "tool"
    
    return END

def call_tool(state):
    print("CALL_TOOL::: ",state)
    result = []
    for tools in state["messages"][-1].tool_calls:
        tool = tools_by_name[tools["name"]]
        obb = tool(tools["args"])
        result.append(ToolMessage(content=obb ,tool_call_id=tools["id"]))

    return {
        "messages" : result
    }


builder.add_node("call_llm", call_llm)
builder.add_node("call_tool", call_tool)


# START INITAILIZING EDGES
builder.add_edge(START, "call_llm")
builder.add_conditional_edges(
    "call_llm",
    should_call_node,
    {
        "tool": "call_tool",
        END: END
    }
)
builder.add_edge("call_tool", "call_llm")

graph = builder.compile()

print("RESULT IS:: ",graph.invoke({"id":"uuid______24234234","messages" : "multipy 5 * 2"})["messages"][-1].content)

CALL_LLM::  {'id': 'uuid______24234234', 'messages': [HumanMessage(content='multipy 5 * 2', additional_kwargs={}, response_metadata={}, id='2f0e5e52-0602-4082-926b-d0f986c84a42')]}
SHOULD_CALL_NODE::  {'messages': [HumanMessage(content='multipy 5 * 2', additional_kwargs={}, response_metadata={}, id='2f0e5e52-0602-4082-926b-d0f986c84a42'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Stg0NS3WAql87f88GS1sNOHj', 'function': {'arguments': '{"a":5,"b":2}', 'name': 'multipy'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 178, 'total_tokens': 198, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-581622cc-0f35-

C:\Users\abdur\AppData\Local\Temp\ipykernel_4052\2439106960.py:27: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  obb = tool(tools["args"])


SHOULD_CALL_NODE::  {'messages': [HumanMessage(content='multipy 5 * 2', additional_kwargs={}, response_metadata={}, id='2f0e5e52-0602-4082-926b-d0f986c84a42'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Stg0NS3WAql87f88GS1sNOHj', 'function': {'arguments': '{"a":5,"b":2}', 'name': 'multipy'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 178, 'total_tokens': 198, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-581622cc-0f35-4feb-b8ba-3273c7d205a8-0', tool_calls=[{'name': 'multipy', 'args': {'a': 5, 'b': 2}, 'id': 'call_Stg0NS3WAql87f88GS1sNOHj', 'type': 'tool_call'}], usage_metadata={'input_tokens': 17